## Cifar-10 TensorFlow2 + Keras CNN 구현

In [12]:
import tensorflow as tf
import numpy as np

In [24]:
EPOCHS = 100
BATCH_SIZE = 128
SHUFFLE = 1000
LR = 1e-4

### 1-데이터 불러오기

In [25]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


### 2-입력데이터 전처리

In [26]:
X_mean = np.mean(X_train, axis=(0, 1, 2))
X_std = np.std(X_train, axis=(0, 1, 2))

X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(35000, 32, 32, 3) (35000, 1) (15000, 32, 32, 3) (15000, 1)


In [28]:
train_tf = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_tf = train_tf.shuffle(SHUFFLE).batch(BATCH_SIZE)

val_tf = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE)

### CNN 모델 빌드

In [29]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras import Model

def my_model():
    inputs = Input(shape=(32, 32, 3))
    
    x = Conv2D(32, (3,3), padding='same', activation='relu')(inputs)
    x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
    x = MaxPool2D((2,2))(x)
    x = Dropout(rate=0.5)(x)
    x = Conv2D(128, (3,3), padding='same', activation='relu')(x)
    x = Conv2D(256, (3,3), padding='valid', activation='relu')(x)
    x = MaxPool2D((2,2))(x)
    x = Dropout(rate=0.5)(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs = inputs, outputs = x)
    
    return model

model = my_model()

### 4-Loss Function & Optimizer

In [30]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

loss_object = SparseCategoricalCrossentropy()
optimizer = Adam(LR)

In [31]:
from tensorflow.keras.metrics import Mean
from tensorflow.keras.metrics import SparseCategoricalAccuracy

train_loss = Mean(name='train_loss')
train_accuracy = SparseCategoricalAccuracy(name='train_accuracy')

val_loss = Mean(name='val_loss')
val_accuracy = SparseCategoricalAccuracy(name='val_accuracy')

### 5-Back Propagation 빌드

In [32]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        outputs = model(images, training=True)
        
        loss = loss_object(labels, outputs)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, outputs)
    
@tf.function
def val_step(images, labels):
    outputs = model(images, training=False)
    v_loss = loss_object(labels, outputs)
    
    val_loss(v_loss)
    val_accuracy(labels, outputs)

### 6-학습하기 Training

In [33]:
for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    val_loss.reset_states()
    val_accuracy.reset_states()
    
    for images, labels in train_tf:
        train_step(images, labels)
        
    for val_images, val_labels in val_tf:
        val_step(val_images, val_labels)
        
    print('Epoch: {:03d}, Train_loss: {:.3f}, Train_acc: {:.3%}, Val_loss: {:.3f}, Val_acc: {:.3%}'.format(
        epoch + 1,
        train_loss.result(),
        train_accuracy.result(),
        val_loss.result(),
        val_accuracy.result()))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch: 001, Train_loss: 1.883, Train_acc: 31.520%, Val_loss: 1.574, Val_acc: 43.913%
Epoch: 002, Train_loss: 1.522, Train_acc: 45.149%, Val_loss: 1.388, Val_acc: 49.607%
Epoch: 003, Train_loss: 1.381, Train_acc: 50.666%, Val_loss: 1.287, Val_acc: 53.487%
Epoch: 004, Train_loss: 1.283, Train_acc: 54.271%, Val_loss: 1.192, Val_acc: 57.347%
Epoch: 005, Train_loss: 1.212, Train_acc: 56.926%, Val_loss: 1.130, Val_acc: 59.793%
Epoch: 006, Train_loss: 1.132, Train_acc: 59.851%, Val_loss: 1.063, Val_acc: 62.147